# NBA Player Performance Analysis - Quick Start

**Goal:** Analyze a player's recent performance and forecast their next season using NBA MCP Synthesis.

**Time:** 10-15 minutes

**Methods Used:**
- Time Series Analysis (ARIMA)
- Trend Detection
- Forecasting with Confidence Intervals
- Real-time Performance Tracking (Particle Filters)

---

## 1. Setup & Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# NBA MCP Synthesis imports
from mcp_server.time_series import TimeSeriesAnalyzer
from mcp_server.particle_filters import PlayerPerformanceParticleFilter

# Plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries loaded successfully!")

## 2. Generate Sample Player Data

In practice, you would load this from your database or CSV file. For this demo, we'll generate realistic sample data.

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Generate 100 games of player data
n_games = 100
dates = pd.date_range(start='2023-10-01', periods=n_games, freq='2D')

# Simulate realistic scoring patterns
base_ppg = 24.5  # Base scoring average
trend = -0.05 * np.arange(n_games)  # Slight decline
seasonality = 2 * np.sin(np.arange(n_games) * 2 * np.pi / 20)  # Hot/cold streaks
noise = np.random.normal(0, 4, n_games)  # Game-to-game variation

points = base_ppg + trend + seasonality + noise
points = np.maximum(points, 5)  # Floor at 5 points

# Create DataFrame
player_data = pd.DataFrame({
    'date': dates,
    'points': points,
    'minutes': np.random.normal(34, 3, n_games),
    'fg_pct': np.random.normal(0.45, 0.05, n_games)
})

print(f"✅ Generated {len(player_data)} games of data")
print(f"\nFirst 5 games:")
player_data.head()

## 3. Visualize Performance Over Time

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))

# Plot actual points
ax.plot(player_data['date'], player_data['points'], 
        'o-', alpha=0.5, label='Game Score', markersize=4)

# Plot 10-game moving average
player_data['ma_10'] = player_data['points'].rolling(window=10).mean()
ax.plot(player_data['date'], player_data['ma_10'], 
        linewidth=3, label='10-Game Average', color='red')

# Add season average line
season_avg = player_data['points'].mean()
ax.axhline(season_avg, color='green', linestyle='--', 
           linewidth=2, label=f'Season Avg: {season_avg:.1f} PPG')

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Points', fontsize=12)
ax.set_title('Player Scoring Performance - 2023-24 Season', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"📊 Season Statistics:")
print(f"   Average: {player_data['points'].mean():.2f} PPG")
print(f"   Std Dev: {player_data['points'].std():.2f}")
print(f"   Max: {player_data['points'].max():.1f}")
print(f"   Min: {player_data['points'].min():.1f}")

## 4. Time Series Analysis

Use TimeSeriesAnalyzer to detect trends and test for stationarity.

In [ ]:
# Create analyzer
ts_analyzer = TimeSeriesAnalyzer(
    data=player_data,
    target_column='points',
    time_column='date'
)

print("="*60)
print("TIME SERIES DIAGNOSTICS")
print("="*60)

# Test for stationarity
adf_result = ts_analyzer.adf_test()
print(f"\n1. Stationarity Test (ADF):")
print(f"   Statistic: {adf_result.statistic:.4f}")
print(f"   P-value: {adf_result.p_value:.4f}")
print(f"   Is Stationary: {adf_result.is_stationary}")

if adf_result.is_stationary:
    print("   ✅ Series is stationary (stable performance)")
else:
    print("   ⚠️  Series is non-stationary (trending)")

# Detect trend
trend_result = ts_analyzer.detect_trend()
print(f"\n2. Trend Detection:")
print(f"   Trend Type: {trend_result.trend_type}")
print(f"   Coefficient: {trend_result.trend_coefficient:.4f} pts/game")
print(f"   P-value: {trend_result.p_value:.4f}")

if abs(trend_result.trend_coefficient) > 0.01 and trend_result.p_value < 0.05:
    direction = "declining" if trend_result.trend_coefficient < 0 else "improving"
    print(f"   📉 Significant {direction} trend detected")
else:
    print(f"   → No significant trend")

## 5. ARIMA Forecasting

Fit an ARIMA model to forecast the next 20 games.

In [ ]:
print("="*60)
print("ARIMA FORECASTING")
print("="*60)

# Fit ARIMA model
arima_result = ts_analyzer.fit_arima(order=(2, 1, 1))
print(f"\nModel: ARIMA(2,1,1)")
print(f"AIC: {arima_result.aic:.2f}")
print(f"BIC: {arima_result.bic:.2f}")

# Forecast next 20 games
forecast = ts_analyzer.forecast(steps=20)

print(f"\nNext 20 Games Forecast:")
print(f"  Average Projected PPG: {forecast.forecast.mean():.2f}")
print(f"  95% Confidence Interval: [{forecast.conf_int_lower.mean():.1f}, {forecast.conf_int_upper.mean():.1f}]")
print(f"  Next Game Prediction: {forecast.forecast.iloc[0]:.1f} points")

# Visualize forecast
fig, ax = plt.subplots(figsize=(14, 6))

# Historical data
ax.plot(player_data['date'], player_data['points'], 
        'o-', alpha=0.6, label='Historical', markersize=3)

# Forecast
forecast_dates = pd.date_range(start=player_data['date'].iloc[-1] + timedelta(days=2), 
                                periods=20, freq='2D')
ax.plot(forecast_dates, forecast.forecast, 
        'ro-', linewidth=2, label='Forecast', markersize=5)

# Confidence interval
ax.fill_between(forecast_dates, 
                 forecast.conf_int_lower, 
                 forecast.conf_int_upper,
                 alpha=0.3, label='95% CI')

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Points', fontsize=12)
ax.set_title('20-Game Forecast with Confidence Intervals', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Particle Filter - Real-Time Performance Tracking

Use a particle filter to track the player's underlying skill and form state.

In [ ]:
print("="*60)
print("PARTICLE FILTER TRACKING")
print("="*60)

# Create particle filter
pf = PlayerPerformanceParticleFilter(
    n_particles=1000,
    skill_volatility=0.05,
    form_persistence=0.7,
    form_volatility=0.2
)

# Filter the season
pf_result = pf.filter_player_season(
    data=player_data.set_index('date'),
    target_col='points'
)

print(f"\n✅ Particle filter completed")
print(f"   Average ESS: {np.mean(pf_result.ess_history):.0f} particles")
print(f"   Resampling Rate: {np.mean(pf_result.resampling_history)*100:.1f}%")

# Visualize skill and form
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Plot 1: Skill trajectory
dates = pf_result.skill_trajectory['date']
skill = pf_result.skill_trajectory['skill']
skill_lower = pf_result.skill_trajectory['skill_lower']
skill_upper = pf_result.skill_trajectory['skill_upper']

ax1.plot(dates, skill, 'b-', linewidth=2, label='Estimated Skill')
ax1.fill_between(dates, skill_lower, skill_upper, alpha=0.3, label='95% CI')
ax1.set_ylabel('Skill Level (log scale)', fontsize=12)
ax1.set_title('Estimated Player Skill Over Time', fontsize=13, fontweight='bold')
ax1.legend()
ax1.grid(alpha=0.3)

# Plot 2: Form (short-term fluctuations)
form = pf_result.form_states['form_mean']
ax2.plot(dates, form, 'r-', linewidth=2, label='Form State')
ax2.axhline(0, color='gray', linestyle='--', alpha=0.5)
ax2.fill_between(dates, form, 0, where=(form > 0), alpha=0.3, color='green', label='Hot')
ax2.fill_between(dates, form, 0, where=(form < 0), alpha=0.3, color='red', label='Cold')
ax2.set_xlabel('Date', fontsize=12)
ax2.set_ylabel('Form State', fontsize=12)
ax2.set_title('Short-Term Form Fluctuations', fontsize=13, fontweight='bold')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Current form assessment
current_form = form.iloc[-1]
print(f"\n📊 Current State Assessment:")
print(f"   Skill Level: {np.exp(skill.iloc[-1]):.1f} pts/game (intrinsic ability)")
print(f"   Form State: {current_form:+.2f}")

if current_form > 0.2:
    print(f"   🔥 Player is currently HOT")
elif current_form < -0.2:
    print(f"   ❄️  Player is currently COLD")
else:
    print(f"   → Player form is neutral")

## 7. Model Validation

Validate the ARIMA model using a train/test split.

In [ ]:
print("="*60)
print("MODEL VALIDATION")
print("="*60)

# Validate forecast with last 15 games as test set
validation = ts_analyzer.validate_forecast(
    order=(2, 1, 1),
    test_size=15
)

print(f"\nForecast Performance (15-game test set):")
print(f"  RMSE: {validation.rmse:.2f} points")
print(f"  MAE: {validation.mae:.2f} points")
print(f"  MAPE: {validation.mape:.1f}%")

# Plot actual vs predicted
fig, ax = plt.subplots(figsize=(12, 6))

test_dates = player_data['date'].iloc[-15:]
ax.plot(test_dates, validation.actual, 'o-', label='Actual', markersize=6, linewidth=2)
ax.plot(test_dates, validation.forecast, 's-', label='Predicted', markersize=6, linewidth=2)

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Points', fontsize=12)
ax.set_title('Model Validation: Actual vs Predicted', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# Accuracy assessment
if validation.mape < 15:
    print("\n✅ Model shows good predictive accuracy (MAPE < 15%)")
elif validation.mape < 25:
    print("\n⚠️  Model shows moderate predictive accuracy (MAPE 15-25%)")
else:
    print("\n❌ Model shows poor predictive accuracy (MAPE > 25%)")

## 8. Summary & Next Steps

### What We Learned:

1. **Time Series Analysis** - Detected trends and tested stationarity
2. **ARIMA Forecasting** - Projected next 20 games with confidence intervals
3. **Particle Filters** - Tracked underlying skill and form states
4. **Model Validation** - Assessed forecast accuracy

### Key Findings:
- Current performance shows [check your output]
- Forecast suggests [check your output]
- Player form is currently [check your output]

### Next Steps:
- Try with real NBA player data
- Compare multiple players using panel data methods
- Add causal analysis for coaching/roster changes
- Explore other notebooks in this series

---

## 📚 Additional Resources

- **[API Reference](../docs/API_REFERENCE.md)** - Complete documentation
- **[Quick Reference](../docs/QUICK_REFERENCE.md)** - Cheat sheet
- **[Complete Tutorial](../docs/tutorials/COMPLETE_WORKFLOW_TUTORIAL.md)** - Full workflow
- **[Getting Started](../docs/GETTING_STARTED.md)** - Installation guide

### Other Notebooks:
- `02_panel_data_comparison.ipynb` - Multi-player comparison
- `03_causal_inference.ipynb` - Treatment effects
- `04_survival_analysis.ipynb` - Career longevity
- `05_real_time_tracking.ipynb` - Live game analytics

---

**🏀 NBA MCP Synthesis - Comprehensive Analytics Made Simple**